In [12]:
import psycopg2
import json 
import geopandas as gpd
import folium
import pandas as pd
from shapely.geometry import Polygon
import geopandas
import ast
from shapely.geometry import Point
import shapely
from math import inf
import geopy.distance as gp

# Часть 1

In [44]:
connection = psycopg2.connect(user="postgres",
                                password="postgres",
                                host="127.0.0.1",
                                port="5432",
                                database="map")                                
cursor = connection.cursor()

In [46]:
cursor.execute("select coordinates from geolocation full outer join object on geolocation.geo_id=object.geo_id where object.type = 1")
record = cursor.fetchall()

In [47]:
object_coordinates = []
for i in range(0, len(record)):
    buff = record[i][0].replace('(', '').replace(')', '')
    buff = [float(val) for val in buff.split(',')]
    object_coordinates.append(buff[::-1]) 

**Открытие файла с сеткой 500 на 500 метров по всей России** 

In [73]:
with open('data/fishnet2021.json') as f: 
    data = json.load(f)

**Создание переменной zid типа список, содержащей список индентификаторов квадратов со сторонами 500 на 500 метров, содержащих информацию о плотности населения**

In [74]:
csv = pd.read_csv("data/01_CLocation_July.csv")
zid = csv['zid'].tolist()

In [76]:
main_table = csv[['zid', 'customers_cnt_home']]
main_table

,zid,customers_cnt_home
0,32909,4
1,32910,25
2,33261,2
3,33262,13
4,33263,10
...,...,...
10235,138913,77
10236,138914,165
10237,138915,587
10238,138916,1206


**Поиск в файле с Сеткой России номера строк, где квадратики есть в файле с плотность населения**

In [77]:
good_i = []
for i in range(0, len(data['features'])):
    if data['features'][i]['properties']['cell_zid'] in zid:
      good_i.append(i)

**Создание столбца с координатами зон плоности населения**

In [ ]:
coordinates = []
for i in good_i:
    coordinates.append(data['features'][i]['geometry']['coordinates'])


main_table['coordinates'] = coordinates

**Создание столбца с координатами зон плоности населения в формате широта/долгота**

In [145]:
coordinates = []
for i in range(0,len(main_table)):
    lol = []
    for j in range(0, len(main_table['coordinates'][i][0])):
        buff = main_table['coordinates'][i][0][j]
        buff = buff[::-1]
        lol.append(buff)
    coordinates.append([lol])

main_table['reverse_coordinates'] = coordinates

**Сохранение датафрейма в формат CSV для дальнейшего ускорения работы**

In [155]:
main_table[['zid', 'customers_cnt_home', 'reverse_coordinates', 'coordinates' ]].to_csv('zones_main_table.csv', sep='\t', encoding='utf-8')

**Отрисовка на карте зон плотности населения и объектов инфраструктуры**

In [ ]:
mmap = folium.Map(location=[55.755819, 37.617644],zoom_start = 11)

for i in range(0,len(main_table['reverse_coordinates'])):
    folium.Polygon(locations=main_table['reverse_coordinates'][i]).add_to(mmap)
for object in object_coordinates:
    folium.Marker(location=object).add_to(mmap) 


mmap

In [54]:
main_table = pd.read_csv('data/zones_main_table.csv', sep='\t')

**Подсчет количества объектов в каждом секторе**

In [ ]:
contains_object = []
for i in range(0, len(main_table)):
    count = 0
    for object in object_coordinates:
        if Polygon(ast.literal_eval(main_table['reverse_coordinates'][i])[0]).contains(Point(object)):
            # Можно удалять элемент, который вошел в полигон, чтобы заново его не проверять
            count +=1
    contains_object.append(count)

**Проверка предудщий функции и вывод максимального количества объектов в одном квадрате**

In [98]:
if len(main_table) == len(contains_object):
    print('all good')
    print('Max in list: ',max(contains_object))
    

all good
Max in list:  15


**Добавление результатов функции в дата фрейм**

In [99]:
main_table['contains_object_4'] = contains_object

**Сохранение финального дата-фрейма в файл .csv**


In [100]:
main_table[['zid','reverse_coordinates', 'customers_cnt_home', 'contains_object_1', 'contains_object_2', 'contains_object_4' ,'contains_object_5', 'contains_object_6',  'contains_object_7']].to_csv('data/table_for_calculation.csv', sep='\t', encoding='utf-8')

# Часть 2. Подсчет процента населения в квадрате 


In [121]:
main_table = pd.read_csv('data/table_for_calculation.csv', sep='\t')

In [122]:
adm_zones = pd.read_csv('data/adm_zid.csv', sep=';')

In [123]:
adm = pd.read_csv('data/01_Location_July.csv', sep=',')

Сначала делаем join отношения квадрата к району, потом приравниаем общую плотность к квадрату. Вычисляем процент количества населения в квадрате к общей количеству населения в районе. 

In [124]:
main_table = main_table.merge(adm_zones['adm_zid'],how='inner', left_on=['zid'], right_on=adm_zones['cell_zid'])
main_table = main_table.merge(adm['customers_cnt_home'],how='inner', left_on=main_table['adm_zid'], right_on=adm['zid'])

part_of_area = []
for i in range(0, len(main_table)):
    part_of_area.append((main_table['customers_cnt_home_x'][i] / main_table['customers_cnt_home_y'][i]) * 100)

main_table['part_of_customer_cnt_home_in_adm_zone'] = part_of_area


Переименовываем новые столбцы

In [131]:
main_table = main_table.rename(columns={'customers_cnt_home_x': 'customers_cnt_home_in_zone', 'customers_cnt_home_y' : 'customers_cnt_home_in_adm_zone' })

Записываем новый в файл

In [136]:
main_table[['zid', 'adm_zid', 'reverse_coordinates' ,'customers_cnt_home_in_zone', 'customers_cnt_home_in_adm_zone', 'contains_object_1', 'contains_object_2', 'contains_object_4', 'contains_object_5', 'contains_object_6', 'contains_object_7', 'part_of_customer_cnt_home_in_adm_zone']].to_csv('data/main_table.csv', sep=';', index=False)

In [138]:
main_table[['reverse_coordinates', 'part_of_customer_cnt_home_in_adm_zone', 'contains_object_1']].sort_values(by=['part_of_customer_cnt_home_in_adm_zone'], ascending=False)

,reverse_coordinates,part_of_customer_cnt_home_in_adm_zone,contains_object_1
13085,"[[[55.98068138176378, 37.17565563034755], [55....",22.543434,0
12043,"[[[55.82388226205482, 37.583871764065705], [55...",22.518980,0
13139,"[[[55.98068138176378, 37.17565563034755], [55....",20.083220,0
11825,"[[[55.81888364387522, 37.86307788584394], [55....",20.016367,0
12044,"[[[55.82387656396234, 37.59185037784273], [55....",19.120571,0
...,...,...,...
4340,"[[[55.4731317417051, 37.17192812230983], [55.4...",0.004488,0
1604,"[[[55.30667834703862, 37.08668340405138], [55....",0.004127,0
1877,"[[[55.34235090609285, 37.01537728462635], [55....",0.004127,0
11994,"[[[55.84605941948398, 37.77551583411016], [55....",0.002466,0
